In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Assuming your dataset has 'Description' and 'Offense' columns
fir = pd.read_csv('C:/Users/Dell/Downloads/fir2dataset2.csv')

# Use LabelEncoder to encode the textual labels
label_encoder = LabelEncoder()

# Fit the label encoder on the labels
label_encoder.fit(fir['Offense'])

# Transform the labels
labels_encoded = label_encoder.transform(fir['Offense'])

In [3]:
fir['Description'] = fir['Description'] + fir['Offense'] + fir['Punishment']
columns_to_drop = ['Cognizable', 'Bailable', 'Court']
firfinal1 = fir.drop(columns=columns_to_drop, axis=1)

firfinal1['New Description'] = firfinal1['Description'] + firfinal1['Offense'] + firfinal1['Punishment']

columns_to_drop = ['Description', 'Offense', 'Punishment']
firfinal2 = firfinal1.drop(columns=columns_to_drop, axis=1)

firfinal2.to_csv('firfinal26.csv', index=False)


In [4]:
firfinal2.head()

,section,New Description
0,140,Description of IPC Section 140\nAccording to s...
1,127,Description of IPC Section 127\nAccording to s...
2,128,Description of IPC Section 128\nAccording to s...
3,129,Description of IPC Section 129\nAccording to s...
4,130,Description of IPC Section 130\nAccording to s...


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can choose a different BERT model if needed
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Example input text
input_text = '''Description of IPC Section 140
According to section 140 of Indian penal code, Whoever, not being a soldier, sailor or airman in the Military, Naval or Air service of the Government of India, wears any garb or carries any token resembling any garb or token used by such a soldier, sailor or airman with the intention that it may be believed that he is such a soldier, sailor or airman, shall be punished with imprisonment of either description for a term which may extend to three months, or with fine which may extend to five hundred rupees, or with both.


IPC 140 in Simple Words
If someone who is not a military member wears a uniform or carries something resembling a military uniform to deceive others into believing they are a soldier, sailor, or airman, they can be punished with up to three months in jail, a fine of up to five hundred rupees, or both.Wearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman3 Months or Fine or BothWearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman3 Months or Fine or BothWearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman3 Months or Fine or Both'''

# Tokenize and format input text for the model
input_ids = tokenizer.encode(input_text, truncation=True, padding=True, return_tensors="pt")

# Make prediction
with torch.no_grad():
    logits = model(input_ids)[0]

# Convert logits to probabilities using softmax
probabilities = torch.nn.functional.softmax(logits, dim=1)

# Get the predicted class (IPC section)
predicted_class = torch.argmax(probabilities).item()

print("Predicted IPC Section:", predicted_class)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted IPC Section: 1


In [6]:
# Convert labels to one-hot encoding
labels_bert = tf.keras.utils.to_categorical(labels_encoded, num_classes=len(label_encoder.classes_))

# Tokenize the data
bert_tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
encodings = bert_tokenizer(list(fir['Description']), truncation=True, padding=True, return_tensors='tf')

In [7]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFBertForSequenceClassification.from_pretrained(model_name, force_download=True, resume_download=False)

config.json: 100%|██████████| 953/953 [00:00<?, ?B/s] 
c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
tf_model

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /nlptown/bert-base-multilingual-uncased-sentiment/cb7ef0d2a4879de37b6a8824b2c6e213526360d4d89b653d674ff58e220161cb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tf_model.h5%3B+filename%3D%22tf_model.h5%22%3B&Expires=1705820822&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTgyMDgyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9ubHB0b3duL2JlcnQtYmFzZS1tdWx0aWxpbmd1YWwtdW5jYXNlZC1zZW50aW1lbnQvY2I3ZWYwZDJhNDg3OWRlMzdiNmE4ODI0YjJjNmUyMTM1MjYzNjBkNGQ4OWI2NTNkNjc0ZmY1OGUyMjAxNjFjYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=vMR76JK1q-JAngy9rGd~-ccwTpdjWqi5YLzSvMQKaHvo-3g2xRLOdbvvrRklEZunwJqHMN1926N9xjafRXOIoW0dtE4n1HtvfYyVJhfMJGC-2gd6Ofej0-BVcVFJ2uT33LRa094EgvCE2cImd3O-hLII3yeEsIdYQVnKvZpRvaXWST3og4LuJQHkn6uND6qdIJ8EbHQBrHZZf1jNg-awucqaunrTnF5YfUyHxOTcY7BduYFvSmQoO~m3ed9RCeDv-

In [ ]:
# Define the BERT model
optimizer_bert = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer_bert, loss=model.compute_loss, metrics=['accuracy'])

NameError: name 'tf' is not defined

In [ ]:
# Train the BERT model
try:
    history_bert = model.fit(encodings, labels_bert, epochs=3, batch_size=8)
except Exception as e:
    print(f"Error during training: {e}")